In [7]:
import pandas as pd

df_processed = pd.read_csv("contrib_from_csv.csv", sep = ",", dtype= str)
df_original = pd.read_csv("contrib_from_original.csv", sep = ",", dtype= str)
df_original = df_original.rename(columns={"info_communes": "joined_id"})
df_original = df_original.rename(columns={"Numéro d'id_cc": "Numéro d'ordre arbitraire"})

In [8]:
df_processed

,Catégorie,Date de réception,Code postal,Code INSEE,Numéro d'ordre arbitraire,Type Graphie TT,Numéro de page,Numéro séquentiel,Contribution,joined_id
0,CC,190322,03350,03048,18236,MD,5,2,-Supprimer la retraite du Président de la Répu...,CC_03350_190322_03048_MD_18236
1,CC,190322,07000,07184,18770,MD,3,1,Imprimé par SOUSA Victor - OH/CCM/GDN\nSujet :...,CC_07000_190322_07184_MD_18770
2,CC,190322,07000,07184,18770,MD,6,2,-Indexation sur l'inflation\nLes salaires de t...,CC_07000_190322_07184_MD_18770
3,CC,190322,07000,07184,18770,MD,7,3,Respecter la parité homme/femme: alignement de...,CC_07000_190322_07184_MD_18770
4,CC,190322,07000,07184,18770,MD,8,4,En annexe\nComment prendre le contrôle d'une N...,CC_07000_190322_07184_MD_18770
...,...,...,...,...,...,...,...,...,...,...
225040,CC,190405,99999,99999,19346,MD,11,5,retraites sont insolvables ou ne retrouvent pa...,CC_99999_190405_99999_MD_19346
225041,CC,190405,99999,99999,19346,MD,13,6,FRANÇAIS DE L'ÉTRANGER...LES GRANDES INÉGALITÉ...,CC_99999_190405_99999_MD_19346
225042,CC,190405,99999,99999,19346,MD,15,7,DEUXIÈME INÉGALITÉ\nAlors que la quasi totalit...,CC_99999_190405_99999_MD_19346
225043,CC,190405,99999,99999,19346,MD,17,8,"Monsieur,\n* Trouver un accord avec l'AEFE par...",CC_99999_190405_99999_MD_19346


In [20]:

#  compter les pages pour chaque cahier dans corpus transcrit

# les numéros de pages ne sont pas dans le bon type (str je crois) --> je les transforme en float, puis en int
df_processed["pages_int"] = df_processed["Numéro de page"].astype(float).astype(int)

# un dataframe qui rassemble le nombre de numéro de pages unique pour chaque cahier - du corpus transcrit
compte = pd.DataFrame({'groupby_par_pages' : df_processed.groupby(["Code INSEE", "Numéro d'ordre arbitraire", "pages_int"]).size()})

# un dataframe qui montre le numéro maximum de pages par cahiers
pages_groupdf = pd.DataFrame({'nombre_de_pages_par_cahier' : df_processed.groupby(["Code INSEE", "Numéro d'ordre arbitraire"])["pages_int"].max()})

# view pages_groupdf
pages_groupdf[0:20]

nombre_de_pages_par_cahier
Code INSEE Numéro d'ordre arbitraire                            
00000      00299                                              11
           00300                                             141
           00490                                               6
           00491                                              41
           01260                                              11
           01603                                              11
           01605                                              13
           01623                                               6
           01690                                              14
           01692                                               9
           01702                                               8
           01744                                               5
           02185                                               5
           02251                                              17
           02651                                               6
           03860                                               7
           03953                                              15
           04597                                               8
           05778                                               9
           06064                                               7

In [26]:
# nombre de pages par communes

# fait un groupby d'abord par code insee, puis par n° de cahier et prend le max de page (n° de page) du sous-ensemble

print("nb de communes ",pages_groupdf.shape)
print("type de numero de communes ",pages_groupdf.dtypes)

# sommes, moyenne, max et min et une occurence du min et du max

print("nombre total de page - sum les pages du groupby pages - insee\n",pages_groupdf["nombre_de_pages_par_cahier"].sum())

print("moyenne les pages du groupby pages - insee",pages_groupdf['nombre_de_pages_par_cahier'].mean())

print("max les pages du groupby pages - insee",pages_groupdf['nombre_de_pages_par_cahier'].max())
print("minimum de pages",pages_groupdf['nombre_de_pages_par_cahier'].min(), "\n")

print("ecart type groupby pages - insee",pages_groupdf['nombre_de_pages_par_cahier'].std(), "\n")

indx_max = pages_groupdf["nombre_de_pages_par_cahier"].astype(float).astype(int).idxmax(axis = 0)
print("\n example du cahier avec le max de page", pages_groupdf.loc[[indx_max]],"\n")
indx_min = pages_groupdf["nombre_de_pages_par_cahier"].astype(float).astype(int).idxmin(axis = 0)
print("exemple d'un cahier avec le min de pages", pages_groupdf.loc[[indx_min]])


nb de communes  (19252, 1)
type de numero de communes  nombre_de_pages_par_cahier    int64
dtype: object
nombre total de page - sum les pages du groupby pages - insee
 372029
moyenne les pages du groupby pages - insee 19.324174111780593
max les pages du groupby pages - insee 1141
minimum de pages 1 

ecart type groupby pages - insee 39.30914345940584 


 example du cahier avec le max de page                                       nombre_de_pages_par_cahier
Code INSEE Numéro d'ordre arbitraire                            
92023      11972                                            1141 

exemple d'un cahier avec le min de pages                                       nombre_de_pages_par_cahier
Code INSEE Numéro d'ordre arbitraire                            
08117      15609                                               1


In [35]:
# liste de code insee unique = liste des communes
uniques_processed = pd.DataFrame(pd.unique(df_processed["Code INSEE"]), columns=["Code INSEE"]).reset_index(drop = True)
print("uniques_processed: ",uniques_processed)
#  nombre de communes
ax = uniques_processed.shape
# total des contributions
ay = df_processed.shape
# moyenne contribution par communes
meanv = ay[0]/ax[0]
print("moyenne contrib / commune",meanv)

uniques_processed:        Code INSEE
0          03048
1          07184
2          07186
3          07340
4          07041
...          ...
16415      97207
16416      97210
16417      97208
16418      97225
16419      99999

[16420 rows x 1 columns]
moyenne contrib / commune 13.705542021924483


In [5]:
# nombre de cahiers par communes pour le processed
# taille des sous-ensemble contrib < cahier < communes
cahiers_communes_groupdf = pd.DataFrame({"count" : df_processed.groupby(["Code INSEE", "Numéro d'ordre arbitraire"]).size()}).reset_index()
print(cahiers_communes_groupdf)
# taille des sous ensemble cahier < commune
cc_groupdf = pd.DataFrame({ "all_count" : cahiers_communes_groupdf.groupby("Code INSEE").size()}).reset_index()

# somme, moy, max, min et leurs occurrences
print("Code INSEE groupby shape : équivalent à nombre de comunes\n", cc_groupdf)
print("somme groupby cahier - insee\n",cc_groupdf['all_count'].sum())

print("moyenne groupby cahier - insee\n",cc_groupdf['all_count'].mean())
print("max les pages du groupby pages - insee\n",cc_groupdf['all_count'].max())

indx_max = cc_groupdf["all_count"].astype(float).astype(int).idxmax(axis = 0)
print(cc_groupdf.iloc[[indx_max]])
print("min les pages du groupby insee",cc_groupdf['all_count'].min())
indx_min = cc_groupdf["all_count"].astype(float).astype(int).idxmin(axis = 0)
print(cc_groupdf.iloc[[indx_min]])

      Code INSEE Numéro d'ordre arbitraire  count
0          00000                     00299      6
1          00000                     00300      5
2          00000                     00490      2
3          00000                     00491     46
4          00000                     01260      4
...          ...                       ...    ...
19247      98612                     17333      2
19248      98613                     16209      4
19249      98817                     16208      7
19250      99999                     19341     19
19251      99999                     19346      9

[19252 rows x 3 columns]
Code INSEE groupby shape : équivalent à nombre de comunes
       Code INSEE  all_count
0          00000         75
1         000000          1
2          01004          1
3          01006          2
4          01009          1
...          ...        ...
16415      97801          1
16416      98612          1
16417      98613          1
16418      98817          1
16419  

In [6]:
# compte le nb de fichier xml par commune pour le corpus original
df_original["xml_as_int"] = df_original["xml"].astype(float).astype(int)
orig_pages_group = pd.DataFrame(df_original.groupby("Code INSEE")["xml_as_int"].sum()).reset_index()

print("size of final concatenated",df_original.shape)

print("sample les pages du groupby pages - insee",orig_pages_group['xml_as_int'].sample(10))
print("moyenne les pages du groupby pages - insee",orig_pages_group['xml_as_int'].mean())
print("max les pages du groupby pages - insee",orig_pages_group['xml_as_int'].max())

indx_max = orig_pages_group["xml_as_int"].astype(float).astype(int).idxmax(axis = 0)
print(orig_pages_group.iloc[[indx_max]])
print("min les pages du groupby insee",orig_pages_group['xml_as_int'].min())
indx_min = orig_pages_group["xml_as_int"].astype(float).astype(int).idxmin(axis = 0)
print(orig_pages_group.iloc[[indx_min]])

size of final concatenated (18854, 13)
sample les pages du groupby pages - insee 9164     12
13252     8
78       28
6085     22
6992     18
13951    10
12651    18
13792     8
12138    16
7428     16
Name: xml_as_int, dtype: int64
moyenne les pages du groupby pages - insee 26.571163021252865
max les pages du groupby pages - insee 3280
  Code INSEE  xml_as_int
0      00000        3280
min les pages du groupby insee 2
     Code INSEE  xml_as_int
8815      48018           2


In [7]:
# compte le nb de cahiers par commune pour le corpus original

orig_cc_group = pd.DataFrame({"count" : df_original.groupby("Code INSEE")["Numéro d'ordre arbitraire"].size()}).reset_index()
print(orig_cc_group.sample(10))

print(orig_cc_group.shape)
print("sommme les cahiers du groupby cahiers - insee",orig_cc_group['count'].sum())

print("moyenne les pages du groupby cahiers - insee",orig_cc_group['count'].mean())
print("max les cahiers du groupby cahiers - insee",orig_cc_group['count'].max())

indx_max = orig_cc_group["count"].astype(float).astype(int).idxmax(axis = 0)
print(orig_cc_group.iloc[[indx_max]])
print("min les cahiers du groupby insee",orig_cc_group['count'].min())
indx_min = orig_cc_group["count"].astype(float).astype(int).idxmin(axis = 0)
print(orig_cc_group.iloc[[indx_min]])

      Code INSEE  count
1076       07329      1
13331      71329      1
2114       14740      1
13433      71564      1
7395       39485      1
846        06002      1
8619       47015      1
7392       39478      1
886        06069      1
14299      81121      1
(16139, 2)
sommme les cahiers du groupby cahiers - insee 18854
moyenne les pages du groupby cahiers - insee 1.1682260363095607
max les cahiers du groupby cahiers - insee 76
  Code INSEE  count
0      00000     76
min les cahiers du groupby insee 1
  Code INSEE  count
1      01004      1


In [10]:
nb_cc = pd.unique(df_original["Numéro d\'ordre arbitraire"])
cahiers = nb_cc.shape[0]
pages = df_original["xml_as_int"].sum()
print("nombre de pages par cahier ORIGINAL xml : ", pages/cahiers)

proc_nb_cc = pd.unique(df_processed["Numéro d\'ordre arbitraire"])
proc_cahiers = proc_nb_cc.shape[0]

print("nombre de pages par cahier PROCESSED csv : ", proc_pages/proc_cahiers)

nombre de pages par cahier ORIGINAL xml :  22.744881722711362
nombre de pages par cahier PROCESSED csv :  19.32618181818182


In [9]:
df_original.loc[df_original["Code INSEE"] == "00000"]

,Unnamed: 0,filename,nb_of_file,xml,categorie_region,joined_id,Catégorie,Code postal,Date de réception,Code INSEE,Type Graphie TT,Numéro d'ordre arbitraire,xml_as_int
0,0,../../../mnt/d/CahiersCitoyensGDN/CC_00/CC_0...,89,44.0,CC_00,CC_00000_190225_00000_MD_00490,CC,00000,190225,00000,MD,00490,44
1,1,../../../mnt/d/CahiersCitoyensGDN/CC_00/CC_0...,89,44.0,CC_00,CC_00000_190225_00000_MD_00491,CC,00000,190225,00000,MD,00491,44
2,2,../../../mnt/d/CahiersCitoyensGDN/CC_00/CC_0...,29,14.0,CC_00,CC_00000_190225_00000_MD_01260,CC,00000,190225,00000,MD,01260,14
3,3,../../../mnt/d/CahiersCitoyensGDN/CC_00/CC_0...,29,14.0,CC_00,CC_00000_190225_00000_MD_01603,CC,00000,190225,00000,MD,01603,14
4,4,../../../mnt/d/CahiersCitoyensGDN/CC_00/CC_0...,37,18.0,CC_00,CC_00000_190225_00000_MD_01605,CC,00000,190225,00000,MD,01605,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16893,1664,../../../mnt/d/CahiersCitoyensGDNpart2/CC_83...,313,156.0,CC_83,CC_83000_190318_00000_MD_19746,CC,83000,190318,00000,MD,19746,156
17181,1952,../../../mnt/d/CahiersCitoyensGDNpart2/CC_85...,57,28.0,CC_85,CC_85000_190315_00000_MD_19605,CC,85000,190315,00000,MD,19605,28
17692,2463,../../../mnt/d/CahiersCitoyensGDNpart2/CC_87...,45,22.0,CC_87,CC_87320_190225_00000_MD_02251,CC,87320,190225,00000,MD,02251,22
18549,3320,../../../mnt/d/CahiersCitoyensGDNpart2/CC_94...,225,112.0,CC_94,CC_94000_190404_00000_MD_19749,CC,94000,190404,00000,MD,19749,112


In [6]:
df_processed.loc[df_processed["Code postal"] == "75020"]

,Catégorie,Date de réception,Code postal,Code INSEE,Numéro d'ordre arbitraire,Type Graphie TT,Numéro de page,Numéro séquentiel,Contribution,joined_id
68817,CC,190313,75020,75120,15886,MD,3,1,LOGOS\nMAIRIE PARIS\nGrand débat national\nCon...,CC_75020_190313_75120_MD_15886
68818,CC,190313,75020,75120,15886,MD,4,2,GRAND DÉBAT\nNATIONAL\nCONTRIBUTIONS ET PROPOS...,CC_75020_190313_75120_MD_15886
68819,CC,190313,75020,75120,15886,MD,5,3,{{Logo}}\nPARIS\nGrand débat national -\nMairi...,CC_75020_190313_75120_MD_15886
68820,CC,190313,75020,75120,15886,MD,6,4,GRAND DÉBAT\nNATIONAL\nCONTRIBUTIONS ET PROPOS...,CC_75020_190313_75120_MD_15886
68821,CC,190313,75020,75120,15886,MD,7,5,{{Logo}}\nPARIS\nGrand débat national\nContrib...,CC_75020_190313_75120_MD_15886
...,...,...,...,...,...,...,...,...,...,...
175286,CC,190320,75020,75120,16403,MD,39,31,Grand débat national Contributions et proposit...,CC_75020_190320_75120_MD_16403
175287,CC,190320,75020,75120,16403,M,40,32,FIN du STATUT de TOUS les FONCTIONNAIRES (verr...,CC_75020_190320_75120_MD_16403
175288,CC,190320,75020,75120,16403,M,42,33,de trop petits moyens Fin des élevages intensi...,CC_75020_190320_75120_MD_16403
175289,CC,190320,75020,75120,16403,MD,43,34,Paris: 26/02/19 Monsieur le Président Je sais ...,CC_75020_190320_75120_MD_16403


In [ ]:
df_original.loc[df_original]